In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [2]:
df = pd.read_csv("/CyberProjects/Phish/Dataset/Final_Data.csv")

In [3]:
df.describe()

,IP_in_Domain,URL_Length,Have_at_Sign,Redirection,Hyphen,URL_Depth,HTTPS/HTTP,Tiny_URL,Domain_Age,Web_Traffic,DNS_Record,Forwarding,iFrame,Label
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.005500,0.773400,0.022600,0.013500,0.093200,3.072000,0.000200,0.090300,0.413700,0.845700,0.100800,0.105300,0.090900,0.500000
std,0.073961,0.418653,0.148632,0.115408,0.290727,2.128631,0.014141,0.286625,0.492521,0.361254,0.301079,0.306955,0.287481,0.500025
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.500000
75%,0.000000,1.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
df.isnull().sum()

Domain          0
IP_in_Domain    0
URL_Length      0
Have_at_Sign    0
Redirection     0
Hyphen          0
URL_Depth       0
HTTPS/HTTP      0
Tiny_URL        0
Domain_Age      0
Web_Traffic     0
DNS_Record      0
Forwarding      0
iFrame          0
Label           0
dtype: int64

In [6]:
df = df.sample(frac=1).reset_index(drop=True)

In [20]:
df = df.drop(['Domain'],axis = 1)

In [21]:
df.head()

,IP_in_Domain,URL_Length,Have_at_Sign,Redirection,Hyphen,URL_Depth,HTTPS/HTTP,Tiny_URL,Domain_Age,Web_Traffic,DNS_Record,Forwarding,iFrame,Label
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,0,1,0,0,0,7,0,0,0,1,0,0,0,0
2,0,1,0,0,0,1,0,0,0,1,0,0,0,0
3,0,1,1,0,0,2,0,0,0,0,0,0,0,1
4,1,0,0,0,0,2,0,0,0,1,0,0,0,1


In [22]:
# Sepratating & assigning features and target columns to X & y
y = df['Label']
X = df.drop('Label',axis=1)
X.shape, y.shape

((10000, 13), (10000,))

In [23]:
# Splitting the dataset into train and test sets: 80-20 split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 12)
X_train.shape, X_test.shape

((8000, 13), (2000, 13))

### Model Training

In [33]:
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss',learning_rate=0.4,max_depth=7)
model.fit(X_train, y_train)

C:\Users\ASUS\anaconda3\envs\Phish\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:50:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.4, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [34]:
# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))


Accuracy: 0.8405
              precision    recall  f1-score   support

           0       0.77      0.98      0.86      1003
           1       0.97      0.70      0.81       997

    accuracy                           0.84      2000
   macro avg       0.87      0.84      0.84      2000
weighted avg       0.87      0.84      0.84      2000



In [37]:
# Save the model to a file named 'model.pkl'
with open('/CyberProjects/Phish/model.pkl', 'wb') as file:
    pickle.dump(model, file)